In [1]:
class TickData(object):
	""" Stores a single unit of data """

	def __init__(self, timestamp='', symbol='',
				open_price=0, close_price=0, total_volume=0):
		self.symbol = symbol
		self.timestamp = timestamp
		self.open_price = open_price
		self.close_price = close_price
		self.total_volume = total_volume

In [2]:
class MarketData(object):
	def __init__(self):
		self.recent_ticks = dict()  # indexed by symbol

	def add_tick_data(self, tick_data):
		self.recent_ticks[tick_data.symbol] = tick_data

	def get_open_price(self, symbol):
		return self.get_tick_data(symbol).open_price

	def get_close_price(self, symbol):
		return self.get_tick_data(symbol).close_price

	def get_tick_data(self, symbol):
		return self.recent_ticks.get(symbol, TickData())

	def get_timestamp(self, symbol):
		return self.recent_ticks[symbol].timestamp

In [12]:
class MarketDataSource(object):
    def __init__(self, symbol, tick_event_handler=None, start='', end=''):
        self.market_data = MarketData()

        self.symbol = symbol
        self.tick_event_handler = tick_event_handler
        self.start, self.end = start, end
        self.df = None

    def fetch_historical_prices(self):
        import pandas_datareader as web
        df = web.DataReader(self.symbol, 'yahoo', self.start, self.end)
        return df

    def run(self):
        if self.df is None:
            self.df = self.fetch_historical_prices()

        total_ticks = len(self.df)
        print('Processing total_ticks:', total_ticks)

        for timestamp, row in self.df.iterrows():
            open_price = row['Open']
            close_price = row['Close']
            volume = row['Volume']

            print(timestamp.date(), 'TICK', self.symbol,
                  'open:', open_price,
                  'close:', close_price)
            tick_data = TickData(timestamp, self.symbol, open_price,
                                close_price, volume)
            self.market_data.add_tick_data(tick_data)

            if self.tick_event_handler:
                self.tick_event_handler(self.market_data)

In [4]:
class Order(object):
	def __init__(self, timestamp, symbol, 
		qty, is_buy, is_market_order, 
		price=0
	):
		self.timestamp = timestamp
		self.symbol = symbol
		self.qty = qty
		self.price = price
		self.is_buy = is_buy
		self.is_market_order = is_market_order
		self.is_filled = False
		self.filled_price = 0
		self.filled_time = None
		self.filled_qty = 0

In [5]:
class Position(object):
	def __init__(self, symbol=''):
		self.symbol = symbol
		self.buys = self.sells = self.net = 0
		self.rpnl = 0
		self.position_value = 0

	def on_position_event(self, is_buy, qty, price):
		if is_buy:
			self.buys += qty
		else:
			self.sells += qty

		self.net = self.buys - self.sells
		changed_value = qty * price * (-1 if is_buy else 1)
		self.position_value += changed_value

		if self.net == 0:
			self.rpnl = self.position_value
			self.position_value = 0

	def calculate_unrealized_pnl(self, price):
		if self.net == 0:
			return 0

		market_value = self.net * price
		upnl = self.position_value + market_value
		return upnl

In [6]:
from abc import abstractmethod

class Strategy:
	def __init__(self, send_order_event_handler):
		self.send_order_event_handler = send_order_event_handler

	@abstractmethod
	def on_tick_event(self, market_data):
		raise NotImplementedError('Method is required!')

	@abstractmethod
	def on_position_event(self, positions):
		raise NotImplementedError('Method is required!')

	def send_market_order(self, symbol, qty, is_buy, timestamp):
		if self.send_order_event_handler:
			order = Order(
				timestamp,
				symbol,
				qty,
				is_buy,
				is_market_order=True,
				price=0,
			)
			self.send_order_event_handler(order)

In [7]:
import pandas as pd

class MeanRevertingStrategy(Strategy):
    def __init__(self, symbol, trade_qty,
        send_order_event_handler=None, lookback_intervals=20,
        buy_threshold=-1.5, sell_threshold=1.5
    ):
        super(MeanRevertingStrategy, self).__init__(
            send_order_event_handler)

        self.symbol = symbol
        self.trade_qty = trade_qty
        self.lookback_intervals = lookback_intervals
        self.buy_threshold = buy_threshold
        self.sell_threshold = sell_threshold

        self.prices = pd.DataFrame()
        self.is_long = self.is_short = False

    def on_position_event(self, positions):
        position = positions.get(self.symbol)

        self.is_long = position and position.net > 0
        self.is_short = position and position.net < 0

    def on_tick_event(self, market_data):
        self.store_prices(market_data)

        if len(self.prices) < self.lookback_intervals:
            return

        self.generate_signals_and_send_order(market_data)

    def store_prices(self, market_data):
        timestamp = market_data.get_timestamp(self.symbol)
        close_price = market_data.get_close_price(self.symbol)
        self.prices.loc[timestamp, 'close'] = close_price

    def generate_signals_and_send_order(self, market_data):
        signal_value = self.calculate_z_score()
        timestamp = market_data.get_timestamp(self.symbol)

        if self.buy_threshold > signal_value and not self.is_long:
            print(timestamp.date(), 'BUY signal')
            self.send_market_order(
                self.symbol, self.trade_qty, True, timestamp)
        elif self.sell_threshold < signal_value and not self.is_short:
            print(timestamp.date(), 'SELL signal')
            self.send_market_order(
                self.symbol, self.trade_qty, False, timestamp)

    def calculate_z_score(self):
        self.prices = self.prices[-self.lookback_intervals:]
        returns = self.prices['close'].pct_change().dropna()
        z_score = ((returns - returns.mean()) / returns.std())[-1]
        return z_score

In [8]:
class BacktestEngine:
	def __init__(self, symbol, trade_qty, start='', end=''):
		self.symbol = symbol
		self.trade_qty = trade_qty
		self.market_data_source = MarketDataSource(
			symbol,
			tick_event_handler=self.on_tick_event,
			start=start, end=end
		)

		self.strategy = None
		self.unfilled_orders = []
		self.positions = dict()
		self.df_rpnl = None
        
	def start(self, **kwargs):
		print('Backtest started...')

		self.unfilled_orders = []
		self.positions = dict()
		self.df_rpnl = pd.DataFrame()

		self.strategy = MeanRevertingStrategy(
			self.symbol,
			self.trade_qty,
			send_order_event_handler=self.on_order_received,
			**kwargs
		)
		self.market_data_source.run()

		print('Backtest completed.')
        
	def on_order_received(self, order):
		""" Adds an order to the order book """
		print(
			order.timestamp.date(),
			'ORDER',
			'BUY' if order.is_buy else 'SELL',
			order.symbol,
			order.qty
		)
		self.unfilled_orders.append(order)
        
	def on_tick_event(self, market_data):
		self.match_order_book(market_data)
		self.strategy.on_tick_event(market_data)
		self.print_position_status(market_data)
        
	def match_order_book(self, market_data):
		if len(self.unfilled_orders) > 0:
			self.unfilled_orders = [
				order for order in self.unfilled_orders
				if self.match_unfilled_orders(order, market_data)
			]
            
	def match_unfilled_orders(self, order, market_data):
		symbol = order.symbol
		timestamp = market_data.get_timestamp(symbol)

		""" Order is matched and filled """
		if order.is_market_order and timestamp > order.timestamp:
			open_price = market_data.get_open_price(symbol)

			order.is_filled = True
			order.filled_timestamp = timestamp
			order.filled_price = open_price

			self.on_order_filled(
				symbol, order.qty, order.is_buy,
				open_price, timestamp
			)
			return False

		return True
    
	def on_order_filled(self, symbol, qty, is_buy, filled_price, timestamp):
		position = self.get_position(symbol)
		position.on_position_event(is_buy, qty, filled_price)
		self.df_rpnl.loc[timestamp, "rpnl"] = position.rpnl

		self.strategy.on_position_event(self.positions)

		print(
			timestamp.date(),
			'FILLED', "BUY" if is_buy else "SELL",
			qty, symbol, 'at', filled_price
		)
        
	def get_position(self, symbol):
		if symbol not in self.positions:
			self.positions[symbol] = Position(symbol)

		return self.positions[symbol]
    
	def print_position_status(self, market_data):
		for symbol, position in self.positions.items():
			close_price = market_data.get_close_price(symbol)
			timestamp = market_data.get_timestamp(symbol)

			upnl = position.calculate_unrealized_pnl(close_price)

			print(
				timestamp.date(),
				'POSITION',
				'value:%.3f' % position.position_value,
				'upnl:%.3f' % upnl,
				'rpnl:%.3f' % position.rpnl
			)

In [17]:
engine = BacktestEngine(
    'ACC.NS', 1,
    start='2015-01-01',
    end='2017-12-31'
)

In [18]:
engine.start(
    lookback_intervals=20,
    buy_threshold=-1.5,
    sell_threshold=1.5
)

Backtest started...
Processing total_ticks: 740
2015-01-01 TICK ACC.NS open: 1400.1500244140625 close: 1403.550048828125
2015-01-02 TICK ACC.NS open: 1406.4000244140625 close: 1425.9000244140625
2015-01-05 TICK ACC.NS open: 1425.0 close: 1433.6500244140625
2015-01-06 TICK ACC.NS open: 1425.0 close: 1383.550048828125
2015-01-07 TICK ACC.NS open: 1382.0 close: 1367.449951171875
2015-01-08 TICK ACC.NS open: 1378.0 close: 1391.4000244140625
2015-01-09 TICK ACC.NS open: 1397.800048828125 close: 1398.5
2015-01-12 TICK ACC.NS open: 1400.0 close: 1404.5999755859375
2015-01-13 TICK ACC.NS open: 1411.4000244140625 close: 1441.699951171875
2015-01-14 TICK ACC.NS open: 1448.0 close: 1478.3499755859375
2015-01-15 TICK ACC.NS open: 1496.5 close: 1514.4000244140625
2015-01-16 TICK ACC.NS open: 1514.5 close: 1518.3499755859375
2015-01-19 TICK ACC.NS open: 1521.9000244140625 close: 1532.9000244140625
2015-01-20 TICK ACC.NS open: 1534.550048828125 close: 1544.300048828125
2015-01-21 TICK ACC.NS open: 15

2015-04-28 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-04-29 TICK ACC.NS open: 1514.5 close: 1484.449951171875
2015-04-29 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-04-30 TICK ACC.NS open: 1480.0999755859375 close: 1433.699951171875
2015-04-30 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-04 TICK ACC.NS open: 1441.0 close: 1474.5999755859375
2015-05-04 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-05 TICK ACC.NS open: 1475.0 close: 1471.4000244140625
2015-05-05 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-06 TICK ACC.NS open: 1466.0 close: 1413.300048828125
2015-05-06 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-07 TICK ACC.NS open: 1414.199951171875 close: 1451.949951171875
2015-05-07 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-08 TICK ACC.NS open: 1455.25 close: 1448.5
2015-05-08 POSITION value:0.000 upnl:0.000 rpnl:-3.950
2015-05-11 TICK ACC.NS open: 1455.0999755859375 close: 1498.949951171875
2015-05-11 SELL signal
2015-05-11 ORDER SELL 

2015-09-15 TICK ACC.NS open: 1385.0 close: 1379.5
2015-09-15 POSITION value:-1359.900 upnl:19.600 rpnl:18.550
2015-09-16 TICK ACC.NS open: 1385.0 close: 1377.300048828125
2015-09-16 POSITION value:-1359.900 upnl:17.400 rpnl:18.550
2015-09-18 TICK ACC.NS open: 1386.0 close: 1375.199951171875
2015-09-18 POSITION value:-1359.900 upnl:15.300 rpnl:18.550
2015-09-21 TICK ACC.NS open: 1364.449951171875 close: 1383.4000244140625
2015-09-21 POSITION value:-1359.900 upnl:23.500 rpnl:18.550
2015-09-22 TICK ACC.NS open: 1389.0 close: 1353.550048828125
2015-09-22 POSITION value:-1359.900 upnl:-6.350 rpnl:18.550
2015-09-23 TICK ACC.NS open: 1335.0 close: 1345.6500244140625
2015-09-23 POSITION value:-1359.900 upnl:-14.250 rpnl:18.550
2015-09-24 TICK ACC.NS open: 1343.0999755859375 close: 1343.050048828125
2015-09-24 POSITION value:-1359.900 upnl:-16.850 rpnl:18.550
2015-09-28 TICK ACC.NS open: 1343.5 close: 1336.75
2015-09-28 POSITION value:-1359.900 upnl:-23.150 rpnl:18.550
2015-09-29 TICK ACC.NS op

2016-02-26 POSITION value:0.000 upnl:0.000 rpnl:26.000
2016-02-29 TICK ACC.NS open: 1189.0 close: 1193.9000244140625
2016-02-29 POSITION value:0.000 upnl:0.000 rpnl:26.000
2016-03-01 TICK ACC.NS open: 1199.0 close: 1232.9000244140625
2016-03-01 SELL signal
2016-03-01 ORDER SELL ACC.NS 1
2016-03-01 POSITION value:0.000 upnl:0.000 rpnl:26.000
2016-03-02 TICK ACC.NS open: 1240.0 close: 1243.25
2016-03-02 FILLED SELL 1 ACC.NS at 1240.0
2016-03-02 POSITION value:1240.000 upnl:-3.250 rpnl:26.000
2016-03-03 TICK ACC.NS open: 1245.699951171875 close: 1254.1500244140625
2016-03-03 POSITION value:1240.000 upnl:-14.150 rpnl:26.000
2016-03-04 TICK ACC.NS open: 1255.0 close: 1239.3499755859375
2016-03-04 POSITION value:1240.000 upnl:0.650 rpnl:26.000
2016-03-08 TICK ACC.NS open: 1239.0999755859375 close: 1241.1500244140625
2016-03-08 POSITION value:1240.000 upnl:-1.150 rpnl:26.000
2016-03-09 TICK ACC.NS open: 1237.300048828125 close: 1243.75
2016-03-09 POSITION value:1240.000 upnl:-3.750 rpnl:26.00

2016-08-09 POSITION value:1418.050 upnl:-267.900 rpnl:17.500
2016-08-10 TICK ACC.NS open: 1688.0 close: 1618.9000244140625
2016-08-10 BUY signal
2016-08-10 ORDER BUY ACC.NS 1
2016-08-10 POSITION value:1418.050 upnl:-200.850 rpnl:17.500
2016-08-11 TICK ACC.NS open: 1620.0 close: 1618.4000244140625
2016-08-11 FILLED BUY 1 ACC.NS at 1620.0
2016-08-11 POSITION value:0.000 upnl:0.000 rpnl:-201.950
2016-08-12 TICK ACC.NS open: 1625.050048828125 close: 1652.5
2016-08-12 POSITION value:0.000 upnl:0.000 rpnl:-201.950
2016-08-16 TICK ACC.NS open: 1654.300048828125 close: 1663.8499755859375
2016-08-16 POSITION value:0.000 upnl:0.000 rpnl:-201.950
2016-08-17 TICK ACC.NS open: 1662.699951171875 close: 1675.0
2016-08-17 POSITION value:0.000 upnl:0.000 rpnl:-201.950
2016-08-18 TICK ACC.NS open: 1682.5 close: 1692.699951171875
2016-08-18 POSITION value:0.000 upnl:0.000 rpnl:-201.950
2016-08-19 TICK ACC.NS open: 1690.0 close: 1691.199951171875
2016-08-19 POSITION value:0.000 upnl:0.000 rpnl:-201.950
20

2017-01-13 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-16 TICK ACC.NS open: 1331.0 close: 1331.0
2017-01-16 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-17 TICK ACC.NS open: 1333.0 close: 1328.9000244140625
2017-01-17 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-18 TICK ACC.NS open: 1337.75 close: 1349.699951171875
2017-01-18 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-19 TICK ACC.NS open: 1351.0 close: 1361.050048828125
2017-01-19 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-20 TICK ACC.NS open: 1357.9000244140625 close: 1326.449951171875
2017-01-20 BUY signal
2017-01-20 ORDER BUY ACC.NS 1
2017-01-20 POSITION value:0.000 upnl:0.000 rpnl:-113.000
2017-01-23 TICK ACC.NS open: 1326.1500244140625 close: 1334.699951171875
2017-01-23 FILLED BUY 1 ACC.NS at 1326.1500244140625
2017-01-23 POSITION value:-1326.150 upnl:8.550 rpnl:-113.000
2017-01-24 TICK ACC.NS open: 1339.0 close: 1358.5999755859375
2017-01-24 POSITION value:-1326.150 upnl:32.450 r

2017-06-22 POSITION value:-1635.000 upnl:16.000 rpnl:92.050
2017-06-23 TICK ACC.NS open: 1653.9000244140625 close: 1636.699951171875
2017-06-23 POSITION value:-1635.000 upnl:1.700 rpnl:92.050
2017-06-27 TICK ACC.NS open: 1641.0 close: 1578.550048828125
2017-06-27 POSITION value:-1635.000 upnl:-56.450 rpnl:92.050
2017-06-28 TICK ACC.NS open: 1573.0 close: 1580.050048828125
2017-06-28 POSITION value:-1635.000 upnl:-54.950 rpnl:92.050
2017-06-29 TICK ACC.NS open: 1591.6500244140625 close: 1579.949951171875
2017-06-29 POSITION value:-1635.000 upnl:-55.050 rpnl:92.050
2017-06-30 TICK ACC.NS open: 1573.25 close: 1568.050048828125
2017-06-30 POSITION value:-1635.000 upnl:-66.950 rpnl:92.050
2017-07-03 TICK ACC.NS open: 1569.9000244140625 close: 1587.5999755859375
2017-07-03 POSITION value:-1635.000 upnl:-47.400 rpnl:92.050
2017-07-04 TICK ACC.NS open: 1591.5999755859375 close: 1580.6500244140625
2017-07-04 POSITION value:-1635.000 upnl:-54.350 rpnl:92.050
2017-07-05 TICK ACC.NS open: 1581.199

2017-11-29 POSITION value:-1709.600 upnl:-27.100 rpnl:32.450
2017-11-30 TICK ACC.NS open: 1674.0999755859375 close: 1667.9000244140625
2017-11-30 POSITION value:-1709.600 upnl:-41.700 rpnl:32.450
2017-12-01 TICK ACC.NS open: 1672.4000244140625 close: 1676.6500244140625
2017-12-01 POSITION value:-1709.600 upnl:-32.950 rpnl:32.450
2017-12-04 TICK ACC.NS open: 1680.0 close: 1694.050048828125
2017-12-04 POSITION value:-1709.600 upnl:-15.550 rpnl:32.450
2017-12-05 TICK ACC.NS open: 1693.0 close: 1698.199951171875
2017-12-05 POSITION value:-1709.600 upnl:-11.400 rpnl:32.450
2017-12-06 TICK ACC.NS open: 1692.0 close: 1690.0999755859375
2017-12-06 POSITION value:-1709.600 upnl:-19.500 rpnl:32.450
2017-12-07 TICK ACC.NS open: 1688.0999755859375 close: 1703.449951171875
2017-12-07 POSITION value:-1709.600 upnl:-6.150 rpnl:32.450
2017-12-08 TICK ACC.NS open: 1708.0 close: 1746.949951171875
2017-12-08 SELL signal
2017-12-08 ORDER SELL ACC.NS 1
2017-12-08 POSITION value:-1709.600 upnl:37.350 rpnl:3